# Yale Entity Resolution: Domain Classification with Mistral AI

This notebook demonstrates how to use **Mistral AI's Classifier Factory** to solve real-world entity disambiguation challenges in library catalog systems. Using Yale University's 17.6M+ catalog records, we'll show how domain classification helps distinguish between entities with identical names but different fields of activity.

## Learning Objectives

By the end of this notebook, you will understand:
1. **Multi-label domain classification**: How to categorize entities across multiple academic and professional domains
2. **Mistral Classifier Factory**: Using state-of-the-art language models for fine-tuned classification tasks  
3. **Entity disambiguation pipeline**: How domain classification resolves the "Franz Schubert problem" (composer vs photographer)
4. **Production deployment**: Building classification systems that scale to millions of library records

## Real-World Challenge: The Franz Schubert Problem

Consider these two Yale catalog records:
- **Franz Schubert** (1797-1828): Famous Austrian composer of classical music
- **Franz Schubert** (20th century): German photographer specializing in archaeological documentation

Without domain classification, entity resolution systems cannot distinguish between these fundamentally different people who happen to share the same name. This notebook shows how Mistral AI helps solve this challenge using semantic understanding of catalog metadata.

## Yale's Domain Taxonomy

Our classification system uses a hierarchical taxonomy with:
- **Parent Categories**: Broad academic divisions (Arts, Sciences, Humanities, etc.)
- **Specific Domains**: Detailed fields like "Music, Sound, and Sonic Arts" vs "Documentary and Technical Arts"
- **Multi-label Support**: Entities can belong to multiple domains (interdisciplinary work)

This approach enables precise entity disambiguation while maintaining the flexibility needed for complex academic and cultural collections.

## Step 1: Install Required Libraries

We need several specialized libraries for this domain classification pipeline:

- **`mistralai`**: Access to Mistral's Classifier Factory for fine-tuning language models on custom datasets
- **`datasets`**: Hugging Face library for loading and managing Yale's training data from their public repository  
- **`wandb`**: Weights & Biases for experiment tracking during model training (integrated with Mistral)
- **`weaviate-client`**: Vector database client for semantic search and similarity operations

These tools work together to create a production-ready classification system that can process millions of catalog records efficiently.

In [4]:
# Install required packages
!pip install mistralai pandas matplotlib seaborn wandb datasets==3.2.0 weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.8/381.8 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.0/433.0 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 31.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.73.0
    Uninstalling grpcio-1.73.0:
      S

In [1]:
import os
from google.colab import userdata
import requests
import json
import random
import time
from typing import Dict, List, Tuple, Any
import hashlib
import pandas as pd
import numpy as np

from openai import OpenAI
from datasets import load_dataset
import weaviate
from weaviate.classes.config import Configure, Property, DataType, VectorDistances
from weaviate.classes.query import MetadataQuery, Filter
from weaviate.util import generate_uuid5
from tqdm import tqdm
RANDOM_SEED = 42

## Step 2: Configure API Keys and Authentication

This step sets up secure access to the services we'll use throughout the classification pipeline:

- **Mistral AI**: For accessing the Classifier Factory, which uses the powerful `ministral-3b-latest` model specifically designed for custom classification tasks
- **OpenAI**: Provides embeddings (`text-embedding-3-small`) used by our Weaviate vector database for semantic search
- **Hugging Face**: Enables us to download Yale's pre-labeled training datasets directly from their public repository
- **Weights & Biases**: Tracks our model training experiments, providing real-time metrics and performance monitoring
- **Weaviate Cloud**: Vector database service for storing and querying entity embeddings at scale

Using Colab's secure `userdata` ensures our API keys remain protected while enabling full access to these production services.

In [2]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')
os.environ["WCD_URL"] = userdata.get('WCD_URL')
os.environ["WCD_GRPC"] = userdata.get('WCD_GRPC')
os.environ["WCD_API_KEY"] = userdata.get('WCD_API_KEY')

## Step 3: Load Yale's Training Datasets

We'll work with two complementary datasets from Yale's entity resolution project:

1. **Training Data**: 2,539 catalog records with bibliographic metadata including titles, subjects, authors, and publication details
2. **Domain Classifications**: Hand-labeled domain assignments for each entity, created by Yale librarians and subject matter experts

### Why These Datasets Matter

Yale's catalog represents one of the world's largest academic collections, with over 17.6 million records spanning centuries of human knowledge. The training data includes challenging disambiguation cases like:

- **Franz Schubert**: Composer vs photographer with identical names
- **Jean Roberts**: Medical researcher vs literary scholar vs political writer  
- **Cross-domain scholars**: Individuals active in multiple academic fields

Each record contains rich contextual information (composite field) that combines title, subjects, and publication details - exactly the kind of semantic context that modern language models excel at understanding.

In [10]:
# Connect to Weaviate
weaviate_api_key = os.environ.get("WCD_API_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")
weaviate_url = os.environ.get("WCD_URL")

openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=weaviate.auth.AuthApiKey(weaviate_api_key),
    headers={"X-OpenAI-Api-Key": openai_api_key}  # For OpenAI vectorizer
)

print("✅ Connected to OpenAI and Weaviate!")

✅ Connected to OpenAI and Weaviate!


In [5]:
# Load from Hugging Face
print("📚 Loading Yale dataset...")
training_data = pd.DataFrame(load_dataset("timathom/yale-library-entity-resolver-training-data")["train"])

print(f"✅ Loaded {len(training_data):,} records")
print(f"   Sample: {training_data.iloc[0]['person']} - {training_data.iloc[0]['title'][:50]}...")

📚 Loading Yale dataset...


(…)ibrary-entity-resolver-training-data.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2539 [00:00<?, ? examples/s]

✅ Loaded 2,539 records
   Sample: Schubert, Franz - Archäologie und Photographie: fünfzig Beispiele ...


In [6]:
# Real Yale entity resolution training data from training_dataset_classified_2025-06-25.csv
yale_catalog_records = [
    # Franz Schubert - Photographer (Documentary Arts)
    {
        "identity": "9.1",
        "personId": "53144#Agent700-22",
        "recordId": "53144",
        "person": "Schubert, Franz",
        "composite": """Title: Archäologie und Photographie: fünfzig Beispiele zur Geschichte und Methode
Subjects: Photography in archaeology
Provision information: Mainz: P. von Zabern, 1978""",
        "title": "Archäologie und Photographie: fünfzig Beispiele zur Geschichte und Methode",
        "subjects": "Photography in archaeology",
        "roles": "Contributor",
        "domain": "Documentary and Technical Arts",
        "marcKey": "7001 $aSchubert, Franz."
    },
    # Franz Schubert - Composer (Music Arts)
    {
        "identity": "9.0",
        "personId": "772230#Agent100-15",
        "recordId": "772230",
        "person": "Schubert, Franz, 1797-1828",
        "composite": """Title: Quartette für zwei Violinen, Viola, Violoncell
Subjects: String quartets--Scores
Provision information: Leipzig: C.F. Peters, [19--?]; Partitur""",
        "title": "Quartette für zwei Violinen, Viola, Violoncell",
        "subjects": "String quartets--Scores",
        "roles": "Contributor",
        "domain": "Music, Sound, and Sonic Arts",
        "marcKey": "1001 $aSchubert, Franz,$d1797-1828."
    },
    # Jean Roberts - Medical Researcher (rich subjects for imputation)
    {
        "identity": "4559.0",
        "personId": "14561127#Agent700-35",
        "recordId": "14561127",
        "person": "Roberts, Jean, 1918-",
        "composite": """Title: Skin conditions and related need for medical care among persons 1-74 years, United States, 1971-1974
Subjects: Skin--Diseases--United States--Statistics; Health surveys--United States; Health surveys; Skin--Diseases; United States
Genres: Statistics
Provision information: Hyattsville, Md: U.S. Department of Health, Education, and Welfare, Public Health Service, Office of the Assistant Secretary for Health, National Center for Health Statistics, 1978""",
        "title": "Skin conditions and related need for medical care among persons 1-74 years, United States, 1971-1974",
        "subjects": "Skin--Diseases--United States--Statistics; Health surveys--United States; Health surveys; Skin--Diseases; United States",
        "roles": "Author",
        "domain": "Medicine, Health, and Clinical Sciences",
        "marcKey": "7001 $aRoberts, Jean,$d1918-$eauthor."
    },
    # Jean Roberts - Literary Scholar (missing subjects - needs imputation!)
    {
        "identity": "4559.2",
        "personId": "1340596#Agent100-17",
        "recordId": "1340596",
        "person": "Roberts, Jean",
        "composite": """Title: Henrik Ibsen's "Peer Gynt": introduction
Subjects: Ibsen, Henrik, 1828-1906. Peer Gynt; Campbell, Duine--Autograph; Roberts, Jean--Autograph
Provision information: [Leicester]: Offcut Private Press, June 26th, 1972""",
        "title": "Henrik Ibsen's \"Peer Gynt\": introduction",
        "subjects": "Ibsen, Henrik, 1828-1906. Peer Gynt; Campbell, Duine--Autograph; Roberts, Jean--Autograph",
        "roles": "Contributor",
        "domain": "Literature and Narrative Arts",
        "marcKey": "1001 $aRoberts, Jean."
    },
    # Jean Roberts - Political Writer (also missing subjects)
    {
        "identity": "4559.1",
        "personId": "2845991#Agent700-18",
        "recordId": "2845991",
        "person": "Roberts, J.E.",
        "composite": """Title: The wise men of Kansas
Subjects: Silver question
Provision information: [Kansas City? Mo.]: [c1896]""",
        "title": "The wise men of Kansas",
        "subjects": "Silver question",
        "roles": "Contributor",
        "domain": "Politics, Policy, and Government",
        "marcKey": "7001 $aRoberts, J.E."
    },
    # Example record with missing subjects (for imputation demo)
    {
        "identity": "demo_missing",
        "personId": "demo#Agent100-99",
        "recordId": "demo999",
        "person": "Roberts, Jean",
        "composite": """Title: Literary analysis techniques in modern drama criticism
Provision information: London: Academic Press, 1975""",
        "title": "Literary analysis techniques in modern drama criticism",
        "subjects": None,  # Missing - we'll impute this!
        "roles": "Author",
        "domain": None,
        "marcKey": "1001 $aRoberts, Jean."
    }
]

df = pd.DataFrame(yale_catalog_records)
print("📚 Real Yale catalog data loaded:")
print(df[['personId', 'person', 'domain', 'title']].to_string())
print(f"\n🔍 Records with missing subjects: {df['subjects'].isna().sum()}")

📚 Real Yale catalog data loaded:
               personId                      person                                   domain                                                                                                 title
0     53144#Agent700-22             Schubert, Franz           Documentary and Technical Arts                            Archäologie und Photographie: fünfzig Beispiele zur Geschichte und Methode
1    772230#Agent100-15  Schubert, Franz, 1797-1828             Music, Sound, and Sonic Arts                                                        Quartette für zwei Violinen, Viola, Violoncell
2  14561127#Agent700-35        Roberts, Jean, 1918-  Medicine, Health, and Clinical Sciences  Skin conditions and related need for medical care among persons 1-74 years, United States, 1971-1974
3   1340596#Agent100-17               Roberts, Jean            Literature and Narrative Arts                                                              Henrik Ibsen's "Peer Gynt": intro

In [11]:
def generate_embedding(text: str, model: str = "text-embedding-3-small") -> np.ndarray:
    """
    Yale's production embedding function from embedding_and_indexing.py

    Args:
        text: Input text to embed
        model: OpenAI embedding model (text-embedding-3-small)

    Returns:
        1536-dimensional embedding vector
    """
    if not text or text.strip() == "":
        # Return zero vector for empty text
        return np.zeros(1536, dtype=np.float32)

    try:
        response = openai_client.embeddings.create(
            model=model,
            input=text
        )

        # Extract embedding from response
        embedding = np.array(response.data[0].embedding, dtype=np.float32)
        return embedding

    except Exception as e:
        print(f"❌ Error generating embedding: {e}")
        return np.zeros(1536, dtype=np.float32)

# Test the embedding function with real Yale data
test_composite = training_data.iloc[0]['composite']
test_embedding = generate_embedding(test_composite)
print(f"✅ Embedding generated successfully! Shape: {test_embedding.shape}")
print(f"   Sample values: {test_embedding[:5]}")
print(f"   Composite text: {test_composite[:80]}...")

✅ Embedding generated successfully! Shape: (1536,)
   Sample values: [ 0.01115062  0.02462124 -0.0213398   0.00958305 -0.04418446]
   Composite text: Title: Archäologie und Photographie: fünfzig Beispiele zur Geschichte und Meth...


In [31]:
def create_entity_schema(client):
    """
    Create EntityString schema
    """
    try:
        # Check if collection already exists
        # Delete existing collection if it exists
        if client.collections.exists("EntityString"):
            client.collections.delete("EntityString")
            print("🗑️ Deleted existing EntityString collection")

        # Create with exact production schema from embedding_and_indexing.py + metadata for imputation
        collection = client.collections.create(
            name="EntityString",
            description="Collection for entity string values with their embeddings",
            vectorizer_config=Configure.Vectorizer.text2vec_openai(
                model="text-embedding-3-small",
                dimensions=1536
            ),
            vector_index_config=Configure.VectorIndex.hnsw(
                ef=128,                    # Production config
                max_connections=64,        # Production config
                ef_construction=128,       # Production config
                distance_metric=VectorDistances.COSINE
            ),
            properties=[
                # Exact production schema
                Property(name="original_string", data_type=DataType.TEXT),
                Property(name="hash_value", data_type=DataType.TEXT),
                Property(name="field_type", data_type=DataType.TEXT),
                Property(name="frequency", data_type=DataType.INT),
                # Added for subject imputation demo
                Property(name="personId", data_type=DataType.TEXT),
                Property(name="recordId", data_type=DataType.TEXT)
            ]
        )

        print("✅ Created EntityString collection with schema")
        return collection

    except Exception as e:
        print(f"❌ Error creating schema: {e}")
        return None

# Create the schema
entity_collection = create_entity_schema(weaviate_client)

🗑️ Deleted existing EntityString collection
✅ Created EntityString collection with schema


In [32]:
def generate_hash(text: str) -> str:
    """
    Generate SHA-256 hash for text (Yale's production method)
    """
    if not text or pd.isna(text):
        return "NULL"
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

# Generate hashes for all fields using Yale's production method
print("🔐 Generating SHA-256 hashes for all records...")

for i, row in training_data.iterrows():
    # Generate hashes for each field type (Yale's approach)
    person_hash = generate_hash(row['person'])
    composite_hash = generate_hash(row['composite'])
    title_hash = generate_hash(row['title'])
    subjects_hash = generate_hash(row['subjects']) if pd.notna(row['subjects']) else "NULL"

    # Store in dataframe
    training_data.at[i, 'person_hash'] = person_hash
    training_data.at[i, 'composite_hash'] = composite_hash
    training_data.at[i, 'title_hash'] = title_hash
    training_data.at[i, 'subjects_hash'] = subjects_hash

print("✅ Generated SHA-256 hashes for all records")
print(f"   Sample person hash: {training_data.iloc[0]['person_hash'][:16]}...")
print(f"   Sample composite hash: {training_data.iloc[0]['composite_hash'][:16]}...")

# Show hash distribution
print(f"\n📊 Hash Statistics:")
print(f"   Unique person hashes: {training_data['person_hash'].nunique()}")
print(f"   Unique composite hashes: {training_data['composite_hash'].nunique()}")
print(f"   NULL subjects hashes: {(training_data['subjects_hash'] == 'NULL').sum()}")

🔐 Generating SHA-256 hashes for all records...
✅ Generated SHA-256 hashes for all records
   Sample person hash: 6cb0f164412941e2...
   Sample composite hash: 324648e06f268fed...

📊 Hash Statistics:
   Unique person hashes: 189
   Unique composite hashes: 2357
   NULL subjects hashes: 351


In [33]:
print("\n🔄 Deduplicating data for indexing...")
unique_objects = []

# Process each field type separately to avoid duplicate UUIDs
field_types = ['person', 'composite', 'title', 'subjects']

for field_type in field_types:
    print(f"   Processing {field_type} field...")

    # Get hash and text columns
    hash_col = f"{field_type}_hash"
    text_col = field_type

    # Skip if field doesn't exist
    if text_col not in training_data.columns:
        continue

    # Filter out NULL hashes and get unique hash-text pairs with metadata
    field_data = training_data[training_data[hash_col] != "NULL"][[hash_col, text_col, 'personId', 'recordId']].drop_duplicates(subset=[hash_col])

    # Add to unique objects with personId and recordId for imputation
    for _, row in field_data.iterrows():
        unique_objects.append({
            'hash_value': row[hash_col],
            'original_string': str(row[text_col]),
            'field_type': field_type,
            'frequency': 1,  # Could be calculated if needed
            'personId': str(row['personId']) if pd.notna(row['personId']) else "",
            'recordId': str(row['recordId']) if pd.notna(row['recordId']) else ""
        })

print(f"✅ Created {len(unique_objects):,} unique objects for indexing")

# Show deduplication statistics
field_counts = {}
for obj in unique_objects:
    field_type = obj['field_type']
    field_counts[field_type] = field_counts.get(field_type, 0) + 1

print(f"\n📊 Unique objects by field type:")
for field_type, count in field_counts.items():
    print(f"   {field_type}: {count:,}")


🔄 Deduplicating data for indexing...
   Processing person field...
   Processing composite field...
   Processing title field...
   Processing subjects field...
✅ Created 6,111 unique objects for indexing

📊 Unique objects by field type:
   person: 189
   composite: 2,357
   title: 1,966
   subjects: 1,599


In [15]:
training_data.head()

,identity,composite,marcKey,person,roles,title,attribution,provision,subjects,genres,relatedWork,recordId,personId,person_hash,composite_hash,title_hash,subjects_hash
0,9.1,Title: Archäologie und Photographie: fünfzig...,"7001 $aSchubert, Franz.","Schubert, Franz",Contributor,Archäologie und Photographie: fünfzig Beispi...,ausgewählt von Franz Schubert und Susanne Gru...,"Mainz: P. von Zabern, 1978",Photography in archaeology,None,None,53144,53144#Agent700-22,6cb0f164412941e2dc71aaeda03a475f6b2b9422bc3b9f...,324648e06f268fed271ca1538b0348e41c6ef387eaa8ca...,db2db2b2b53b9ec2965192ee93769a30212f070f091fdf...,40d3e3ee1b9a90f415443faf020f58faddc7aba584006d...
1,9.0,"Title: Quartette für zwei Violinen, Viola, Vi...","1001 $aSchubert, Franz,$d1797-1828.","Schubert, Franz, 1797-1828",Contributor,"Quartette für zwei Violinen, Viola, Violoncell",von Franz Schubert,"Leipzig: C.F. Peters, [19--?] Partitur",String quartets--Scores,None,None,772230,772230#Agent100-15,71cc57bae228d21e11cc583581e32ca275592c29549c7a...,8c79ba57383510bd5bc24da3e082bb72e54ef38f5c94ef...,347762c310bbf840b99294d0deeda46aeaff2ad8278973...,33a5a7fe06a8471ea06c499372bc31efad651a5059d914...
2,9.0,"Title: Quartette für zwei Violinen, Viola, Vi...","1001 $aSchubert, Franz,$d1797-1828.","Schubert, Franz, 1797-1828",Contributor,"Quartette für zwei Violinen, Viola, Violoncell",von Franz Schubert,"Leipzig: C.F. Peters, [19--?] Partitur",String quartets--Scores,None,"Quartets, violins (2), viola, cello. Selections",772230,772230#Hub240-16-Agent,71cc57bae228d21e11cc583581e32ca275592c29549c7a...,6d24c4811bf8e64c9917b21e9dd636f02aaf92407f2bb1...,347762c310bbf840b99294d0deeda46aeaff2ad8278973...,33a5a7fe06a8471ea06c499372bc31efad651a5059d914...
3,9.0,Title: Der Hirt auf dem Felsen: nach Wilh. Mü...,"1001 $aSchubert, Franz,$d1797-1828.","Schubert, Franz, 1797-1828",Contributor,Der Hirt auf dem Felsen: nach Wilh. Müllers G...,Franz Schubert,"Wiesbaden: Breitkopf & Härtel, [19--]","Müller, Wilhelm, 1794-1827; Songs (High voice...",Songs,None,666968,666968#Agent100-16,71cc57bae228d21e11cc583581e32ca275592c29549c7a...,54ea0f1d21cfd01163fe5d489588df298bc841f5dbdde0...,eeaf3b4006575d9ad07439336ca175fd7b7148909a2a30...,adf321cfacc01077064806fb37f16427792a5a604d8cf6...
4,9.0,"Title: Quintett in A für Klavier, Violine, Vi...","1001 $aSchubert, Franz,$d1797-1828.","Schubert, Franz, 1797-1828",Contributor,"Quintett in A für Klavier, Violine, Viola, Vi...",Franz Schubert ; herausgegeben von Arnold Feil,"Kassel; New York: Bärenreiter, 1987, c1975","Quintets (Piano, violin, viola, cello, double ...",None,None,786540,786540#Agent100-16,71cc57bae228d21e11cc583581e32ca275592c29549c7a...,8a73e2f764e10351761407869a9eb8cb981cf35ba777c7...,1ff138d7100c9f1f3db5a1d45a291a6a5d7b955be2f1b3...,b75cd0573999d4ca79bc8ab8eb31349c7d028a19894161...


In [34]:
def index_entities(collection, dataframe):
    """
    Index Yale entity strings in Weaviate
    """
    print("🔄 Indexing Yale entity strings in Weaviate...")

    indexed_count = 0
    batch_size = 100

    print("🚀 Indexing deduplicated data...")

    with collection.batch.dynamic() as batch:
        for obj in tqdm(unique_objects, desc="Indexing unique objects"):
            try:
                # Generate UUID using production method (hash + field_type)
                uuid_input = f"{obj['hash_value']}_{obj['field_type']}"
                uuid = generate_uuid5(uuid_input)

                # Add to batch
                batch.add_object(
                    uuid=uuid,
                    properties={
                        "original_string": obj['original_string'],
                        "hash_value": obj['hash_value'],
                        "field_type": obj['field_type'],
                        "frequency": obj['frequency'],
                        "personId": obj['personId'],
                        "recordId": obj['recordId']
                    }
                )
                indexed_count += 1

            except Exception as e:
                print(f"❌ Error indexing {obj['field_type']}: {e}")

    print(f"✅ Successfully indexed {indexed_count:,} unique objects")

    return indexed_count

# Index our real Yale data
indexed_count = index_entities(entity_collection, training_data)

# Verify indexing
print(f"\n🔍 Verification:")
print(f"   Expected records: {len(training_data) * 3 + training_data['subjects'].notna().sum()}")  # person + composite + title + subjects (if not null)
print(f"   Actually indexed: {indexed_count}")

🔄 Indexing Yale entity strings in Weaviate...
🚀 Indexing deduplicated data...


Indexing unique objects: 100%|██████████| 6111/6111 [00:15<00:00, 400.08it/s]


✅ Successfully indexed 6,111 unique objects

🔍 Verification:
   Expected records: 9805
   Actually indexed: 6111


In [31]:
# Test semantic search
print("🔍 Testing semantic search...")
query = "classical compositions"

# Search
search_results = entity_collection.query.near_text(
    query=query,
    limit=5,
    return_properties=["original_string", "field_type", "hash_value"],
    return_metadata=["distance"]
)

print(f'\n🎼 Search results for "{query}":')
for i, obj in enumerate(search_results.objects, 1):
    props = obj.properties
    distance = obj.metadata.distance
    cosine_similarity = 1 - distance  # Convert distance to cosine similarity

    print(f"   {i}. {props['field_type']}: {props['original_string'][:60]}...")
    print(f"      Cosine Similarity: {cosine_similarity:.4f}")

# Check counts by field type
print(f"\n📊 Objects by field type:")
for field_type in ["person", "composite", "title", "subjects"]:
    from weaviate.classes.query import Filter
    result = entity_collection.aggregate.over_all(
        filters=Filter.by_property("field_type").equal(field_type),
        total_count=True
    )
    print(f"   {field_type}: {result.total_count:,}")

# Total count
result = entity_collection.aggregate.over_all(total_count=True)
print(f"\n📊 Total indexed: {result.total_count:,} objects")

🔍 Testing semantic search...

🎼 Search results for "classical compositions":
   1. subjects: Piano quartets; Piano quintets; Piano trios; Sonatas (Violin...
      Cosine Similarity: 0.4609
   2. subjects: Concertos (Piano); Sonatas (Violin and piano)...
      Cosine Similarity: 0.4499
   3. composite: Title: Piano sonatas: D 557, D 575, D 894
Version of: Sonata...
      Cosine Similarity: 0.4492
   4. subjects: Sonatas (Cello and piano); Piano music...
      Cosine Similarity: 0.4479
   5. title: Piano sonatas: D 557, D 575, D 894...
      Cosine Similarity: 0.4458

📊 Objects by field type:
   person: 189
   composite: 2,357
   title: 1,966
   subjects: 1,599

📊 Total indexed: 6,111 objects


In [40]:
print("🎯 YALE SUBJECT IMPUTATION DEMONSTRATION")
print("=" * 50)
print("We'll demonstrate how Yale's hot-deck imputation works using semantic similarity")
print("to find appropriate subjects for records that are missing subject information.\n")

# Step 1: Introduce our target record (missing subjects)
print("📖 STEP 1: Our Target Record (Missing Subjects)")
print("-" * 45)
target_record = {
    "personId": "demo#Agent100-99",
    "person": "Roberts, Jean",
    "composite": "Title: Literary analysis techniques in modern drama criticism\\nProvision information: London: Academic Press, 1975",
    "title": "Literary analysis techniques in modern drama criticism",
    "subjects": None  # ← This is what we want to impute!
}

print(f"   📋 PersonId: {target_record['personId']}")
print(f"   👤 Person: {target_record['person']}")
print(f"   📚 Title: {target_record['title']}")
print(f"   📄 Composite: {target_record['composite']}")
print(f"   ❌ Subjects: None (this is what we need to find!)")

🎯 YALE SUBJECT IMPUTATION DEMONSTRATION
We'll demonstrate how Yale's hot-deck imputation works using semantic similarity
to find appropriate subjects for records that are missing subject information.

📖 STEP 1: Our Target Record (Missing Subjects)
---------------------------------------------
   📋 PersonId: demo#Agent100-99
   👤 Person: Roberts, Jean
   📚 Title: Literary analysis techniques in modern drama criticism
   📄 Composite: Title: Literary analysis techniques in modern drama criticism\nProvision information: London: Academic Press, 1975
   ❌ Subjects: None (this is what we need to find!)


In [48]:
print("🔍 STEP 2: Finding Similar Records")
print("-" * 35)
print("We search for composite texts that are semantically similar to our target...")
print(f"   🎯 Query: '{target_record['composite']}'")
print()

similar_composites = entity_collection.query.near_text(
    query=target_record['composite'],
    filters=Filter.by_property("field_type").equal("composite"),
    limit=8,
    return_properties=["original_string", "personId", "recordId"],
    return_metadata=MetadataQuery(distance=True)
)

print(f"   📊 Found {len(similar_composites.objects)} similar composite records:")
# Show the records we found
for i, obj in enumerate(similar_composites.objects, 1):
    similarity = 1.0 - obj.metadata.distance
    print(f"      {i}. Similarity: {similarity:.3f} - {obj.properties['original_string'][:70]}...")

🔍 STEP 2: Finding Similar Records
-----------------------------------
We search for composite texts that are semantically similar to our target...
   🎯 Query: 'Title: Literary analysis techniques in modern drama criticism\nProvision information: London: Academic Press, 1975'

   📊 Found 8 similar composite records:
      1. Similarity: 0.500 - Title: Dramatic Annals: Critiques on Plays and Performances. Vol 1. 17...
      2. Similarity: 0.479 - Title: The Modern Theatre; A Collection of Successful Modern Plays, As...
      3. Similarity: 0.450 - Title: Playhouses, Theatres and Other Places of Public Amusement in Lo...
      4. Similarity: 0.445 - Title: The Critic; or, A Tragedy Rehears'd
Subjects: Celebrity Culture...
      5. Similarity: 0.438 - Title: The saving lie: Harold Bloom and deconstruction
Subjects: Criti...
      6. Similarity: 0.423 - Title: Metalinguagem: ensaios de teoria e crítica literária
Subjects...
      7. Similarity: 0.421 - Title: Opinions and perspectives fro

In [44]:
# Step 3: Show candidate records and their similarity scores
print("📋 STEP 3: Candidate Records with Similarity Scores")
print("-" * 50)
candidates_with_subjects = []

for i, obj in enumerate(similar_composites.objects, 1):
    similarity = 1.0 - obj.metadata.distance
    person_id = obj.properties["personId"]
    record_id = obj.properties["recordId"]
    composite_text = obj.properties["original_string"]

    print(f"   {i}. Similarity: {similarity:.3f}")
    print(f"      PersonId: {person_id}")
    print(f"      Composite: {composite_text[:80]}...")

    # Check if this person has subjects (potential donor)
    subject_query = entity_collection.query.fetch_objects(
        filters=(
            Filter.by_property("personId").equal(person_id) &
            Filter.by_property("field_type").equal("subjects")
        ),
        return_properties=["original_string"],
        limit=1
    )

    if subject_query.objects:
        subject_text = subject_query.objects[0].properties["original_string"]
        print(f"      ✅ Has Subjects: {subject_text[:60]}...")
        candidates_with_subjects.append({
            'personId': person_id,
            'recordId': record_id,
            'similarity': similarity,
            'subjects': subject_text,
            'composite': composite_text
        })
    else:
        print(f"      ❌ No Subjects: Cannot use as donor")

📋 STEP 3: Candidate Records with Similarity Scores
--------------------------------------------------
   1. Similarity: 0.500
      PersonId: 13930523#Agent100-10
      Composite: Title: Dramatic Annals: Critiques on Plays and Performances. Vol 1. 1741-1785. C...
      ✅ Has Subjects: Celebrity Culture & Fashion; Business & Finance; Modes of Pe...
   2. Similarity: 0.479
      PersonId: 13933294#Agent700-39
      Composite: Title: The Modern Theatre; A Collection of Successful Modern Plays, As Acted at ...
      ✅ Has Subjects: Modes of Performance: Costume, Scenography & Spectacle; Cove...
   3. Similarity: 0.450
      PersonId: 13930526#Agent700-57
      Composite: Title: Playhouses, Theatres and Other Places of Public Amusement in London and i...
      ✅ Has Subjects: Celebrity Culture & Fashion; Business & Finance; Modes of Pe...
   4. Similarity: 0.445
      PersonId: 13932650#Agent100-10
      Composite: Title: The Critic; or, A Tragedy Rehears'd
Subjects: Celebrity Culture & Fas

In [47]:
print("📊 STEP 4: Understanding Similarity Scores")
print("-" * 42)
print(f"   🎯 Found {len(candidates_with_subjects)} potential donor records")
print("   📏 Similarity scores range from 0.0 (different) to 1.0 (identical)")
print("   🚪 Yale's threshold: 0.45 (only use candidates above this)")
print()

# Filter candidates by threshold
threshold = 0.45
good_candidates = [c for c in candidates_with_subjects if c['similarity'] >= threshold]
print(f"   ✅ Candidates above threshold ({threshold}): {len(good_candidates)}")

if good_candidates:
    print("   🏆 Best candidates for subject imputation:")
    for i, candidate in enumerate(good_candidates[:3], 1):
        print(f"      {i}. Similarity {candidate['similarity']:.3f}: {candidate['subjects'][:500]}...")
else:
    print("   ⚠️  No candidates above threshold - imputation not recommended")

📊 STEP 4: Understanding Similarity Scores
------------------------------------------
   🎯 Found 8 potential donor records
   📏 Similarity scores range from 0.0 (different) to 1.0 (identical)
   🚪 Yale's threshold: 0.45 (only use candidates above this)

   ✅ Candidates above threshold (0.45): 3
   🏆 Best candidates for subject imputation:
      1. Similarity 0.500: Celebrity Culture & Fashion; Business & Finance; Modes of Performance: Costume, Scenography & Spectacle; Women in Eighteenth Century Drama; Theatre Royal Drury Lane; Covent Garden Theatre; Goodman's Fields; Richmond Theatre; The Little Theatre (or Theatre Royal), Haymarket; Royalty Theatre; Garrick, David; Barry, Elizabeth; Fenton, Lavinia; Walker, Thomas; Pinkethman, William; Cibber, Colley; Cibber, Susannah; Pritchard, Mrs; Clive, Catherine; Woodward, Henry; Foote, Samuel; King, Thomas; Reddis...
      2. Similarity 0.479: Modes of Performance: Costume, Scenography & Spectacle; Covent Garden Theatre; The Little Theatre (or 

In [49]:
# Step 5: Demonstrate the hot-deck imputation process
print("🧮 STEP 5: Hot-Deck Imputation Process")
print("-" * 40)
if good_candidates:
    print("   🔄 Yale's weighted centroid algorithm:")
    print("      1. Weight each candidate by similarity score")
    print("      2. Calculate centroid of subject embeddings")
    print("      3. Find subject closest to the centroid")
    print()

    # Simple demonstration (using similarity-weighted selection)
    best_candidate = max(good_candidates, key=lambda x: x['similarity'])
    confidence = best_candidate['similarity'] * 0.85  # Approximate confidence calculation

    print(f"   🎯 Selected Subject (highest similarity):")
    print(f"      📝 Subject: {best_candidate['subjects']}")
    print(f"      📊 Source Similarity: {best_candidate['similarity']:.3f}")
    print(f"      🎪 Confidence Score: {confidence:.3f}")
    print(f"      📋 Source PersonId: {best_candidate['personId']}")

🧮 STEP 5: Hot-Deck Imputation Process
----------------------------------------
   🔄 Yale's weighted centroid algorithm:
      1. Weight each candidate by similarity score
      2. Calculate centroid of subject embeddings
      3. Find subject closest to the centroid

   🎯 Selected Subject (highest similarity):
      📝 Subject: Celebrity Culture & Fashion; Business & Finance; Modes of Performance: Costume, Scenography & Spectacle; Women in Eighteenth Century Drama; Theatre Royal Drury Lane; Covent Garden Theatre; Goodman's Fields; Richmond Theatre; The Little Theatre (or Theatre Royal), Haymarket; Royalty Theatre; Garrick, David; Barry, Elizabeth; Fenton, Lavinia; Walker, Thomas; Pinkethman, William; Cibber, Colley; Cibber, Susannah; Pritchard, Mrs; Clive, Catherine; Woodward, Henry; Foote, Samuel; King, Thomas; Reddish, Samuel; Quick, John; Barry, Spranger; Mattocks, Mrs; Miss Younge; Dibdin, Charles; Abington, Frances; Lewis, Charles Lee; Sheridan, Thomas; Cowley, Hannah; Mr Aickin; S

In [50]:
# Close connection when done
weaviate_client.close()

In [ ]:
# Load datasets
training_data = pd.DataFrame(load_dataset("timathom/yale-library-entity-resolver-training-data")["train"])
classifications = pd.DataFrame(load_dataset("timathom/yale-library-entity-resolver-classifications")["train"])

In [ ]:
if training_data is not None and not training_data.empty \
and classifications is not None and not classifications.empty:
    print("📚 Loaded Labeled Datasets")
    print("=" * 50)

    print(f"\nTraining DataFrame has {len(training_data)} rows\n")
    first_row_training = training_data.iloc[0]
    print("First row:")
    print(first_row_training)

    print(f"Classification DataFrame has {len(classifications)} rows\n")
    first_row_class = classifications.iloc[0]
    print("First row:")
    print(first_row_class)

📚 Loaded Labeled Datasets

Training DataFrame has 2539 rows

First row:
identity                                                     9.1
composite      Title: Archäologie und Photographie: fünfzig...
marcKey                                  7001 $aSchubert, Franz.
person                                           Schubert, Franz
roles                                                Contributor
title          Archäologie und Photographie: fünfzig Beispi...
attribution    ausgewählt von Franz Schubert und Susanne Gru...
provision                             Mainz: P. von Zabern, 1978
subjects                              Photography in archaeology
genres                                                      None
relatedWork                                                 None
recordId                                                   53144
personId                                       53144#Agent700-22
Name: 0, dtype: object
Classification DataFrame has 2539 rows

First row:
personId 

## Step 4: Initialize Mistral Classifier Factory

### What is Mistral's Classifier Factory?

Mistral's Classifier Factory represents a breakthrough in custom classification tasks. Instead of training models from scratch or using generic pre-trained classifiers, it fine-tunes the powerful `ministral-3b-latest` model specifically on your labeled data.

### Key Advantages for Entity Resolution:

1. **Semantic Understanding**: Unlike traditional keyword-based approaches, Mistral understands the meaning behind catalog metadata
2. **Multi-label Support**: Can assign entities to multiple domains simultaneously (essential for interdisciplinary scholars)
3. **Few-shot Learning**: Achieves high accuracy with relatively small training datasets (our 2,539 examples)
4. **Production Ready**: Automatically scales to handle millions of classifications with consistent performance

### Why Fine-tuning Beats Base Models

Generic language models struggle with domain-specific classification because they lack specialized knowledge about academic taxonomies and library catalog structures. Fine-tuning teaches the model to recognize patterns specific to Yale's classification system, dramatically improving accuracy on real-world entity disambiguation tasks.

In [ ]:
# Initialize Mistral client
client = Mistral(api_key=os.environ['MISTRAL_API_KEY'])
print("🤖 Mistral client initialized")

🤖 Mistral client initialized


## Step 5: Prepare Multi-label Training Data

### Understanding Multi-label Classification

Traditional classification assigns each item to exactly one category. Multi-label classification allows items to belong to multiple categories simultaneously - essential for academic entities who often work across disciplines.

For example, a scholar studying "computational approaches to medieval literature" might be classified as:
- **Primary Domain**: Literature and Narrative Arts  
- **Secondary Domain**: Computer Science and Information Technology
- **Parent Categories**: Both "Humanities" and "Sciences"

### Data Format for Mistral

Mistral expects training data in a specific JSON format where each example contains:
- **Text**: The composite catalog metadata (title + subjects + publication info)
- **Labels**: A dictionary with multiple classification targets:
  - `domain`: Specific academic fields (can be multiple)
  - `parent_category`: Broader disciplinary groupings

This format enables the model to learn both fine-grained and hierarchical classification patterns simultaneously.

In [ ]:
# Create entity lookup
entity_lookup = {}
for _, row in training_data.iterrows():
    person_id = str(row['personId'])
    entity_lookup[person_id] = row['composite']

print(f"Created entity lookup for {len(entity_lookup)} entities")

Created entity lookup for 2539 entities


In [ ]:
# Convert to Mistral format
training_examples = []

for idx, row in classifications.iterrows():
    person_id = row.get('personId', idx)  # Use personId column or index

    # Get composite text
    composite_text = entity_lookup.get(person_id)
    if not composite_text:
        continue

    # Extract labels and parent categories
    labels_list = row.get('label', [])
    paths_list = row.get('path', [])

    if not labels_list:
        continue

    # Extract parent categories from paths
    parent_categories = []
    for path in paths_list:
        if " > " in path:
            parent_categories.append(path.split(" > ")[0])

    # Create training example in Mistral format
    training_examples.append({
        "text": composite_text,
        "labels": {
            "domain": labels_list,  # Multi-label list
            "parent_category": parent_categories
        }
    })

print(f"Created {len(training_examples)} training examples")

# Show sample
print("\n📝 Sample training example:")
sample_ex = training_examples[0]
print(f"Text: {sample_ex['text'][:500]}")
print(f"Domains: {sample_ex['labels']['domain']}")
print(f"Parents: {sample_ex['labels']['parent_category']}")

Created 2539 training examples

📝 Sample training example:
Text: Title: Archäologie und Photographie: fünfzig Beispiele zur Geschichte und Methode
Subjects: Photography in archaeology
Provision information: Mainz: P. von Zabern, 1978
Domains: ['Documentary and Technical Arts', 'History, Heritage, and Memory']
Parents: ['Arts, Culture, and Creative Expression', 'Humanities, Thought, and Interpretation']


## Step 6: Create Training and Validation Splits

### Why Data Splitting Matters

Proper data splitting is crucial for reliable model evaluation:

- **Training Set (80%)**: Used to teach the model classification patterns
- **Validation Set (20%)**: Used to evaluate performance on unseen data during training
- **Random Shuffling**: Ensures both sets represent the full range of domains and difficulty levels

### Best Practices

We use a fixed random seed (42) to ensure reproducible results across different training runs. This is essential for:
- **Scientific reproducibility**: Others can replicate our exact results
- **Model comparison**: Fair evaluation of different approaches
- **Debugging**: Consistent results help identify issues in the pipeline

The 80/20 split provides enough training data for effective learning while reserving sufficient examples for robust validation metrics.

In [ ]:
# Split data (80% train, 20% validation)
random.seed(42)
random.shuffle(training_examples)

split_idx = int(len(training_examples) * 0.8)
train_examples = training_examples[:split_idx]
val_examples = training_examples[split_idx:]

print(f"Training set: {len(train_examples)} examples")
print(f"Validation set: {len(val_examples)} examples")

Training set: 2031 examples
Validation set: 508 examples


## Step 7: Export Data in JSON Lines Format

### Understanding JSON Lines (JSONL)

JSONL is the standard format for machine learning training data:
- **One example per line**: Each line contains a complete training example
- **Streaming friendly**: Can process large datasets without loading everything into memory
- **Platform standard**: Used by most ML services including Mistral, OpenAI, and Hugging Face

### Why We Save Locally First

Before uploading to Mistral's servers, we save the data locally to:
1. **Verify format**: Check that our data transformation worked correctly
2. **Enable debugging**: Inspect examples if training fails
3. **Create backups**: Preserve our processed data for future use
4. **Cost management**: Avoid repeated processing charges if uploads fail

In [ ]:
# Save to JSONL files
def save_jsonl(examples, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for example in examples:
            f.write(json.dumps(example, ensure_ascii=False) + '\n')
    print(f"Saved {len(examples)} examples to {filepath}")

os.makedirs("mistral", exist_ok=True)

train_path = "./mistral/mistral_train_2025-07-01.jsonl"
val_path = "./mistral/mistral_val_2025-07-01.jsonl"

save_jsonl(train_examples, train_path)
save_jsonl(val_examples, val_path)

print("✅ Data preparation complete!")

Saved 2031 examples to ./mistral/mistral_train_2025-07-01.jsonl
Saved 508 examples to ./mistral/mistral_val_2025-07-01.jsonl
✅ Data preparation complete!


## Step 8: Upload Training Data to Mistral

### The Upload Process

Mistral's Classifier Factory requires training data to be uploaded to their secure servers before fine-tuning can begin. This process:

1. **Validates format**: Ensures our JSONL data meets Mistral's requirements
2. **Assigns file IDs**: Creates unique identifiers for tracking our datasets
3. **Enables versioning**: Allows us to reference specific data versions in training jobs
4. **Provides security**: Data is encrypted and access-controlled on Mistral's infrastructure

### What Happens Next

Once uploaded, these file IDs become the inputs to our fine-tuning job. Mistral will:
- **Parse the data**: Extract training examples and labels
- **Balance classes**: Handle any imbalances in domain representation
- **Create batches**: Organize data for efficient GPU training
- **Track progress**: Monitor training metrics in real-time

In [ ]:
# Upload the training data
print("📤 Uploading training data...")
training_data = client.files.upload(
    file={
        "file_name": "mistral_train_2025-07-01.jsonl",
        "content": open(train_path, "rb"),
    }
)
print(f"✅ Training file uploaded: {training_data.id}")

# Upload the validation data
print("📤 Uploading validation data...")
validation_data = client.files.upload(
    file={
        "file_name": "mistral_val_2025-07-01.jsonl",
        "content": open(val_path, "rb"),
    }
)
print(f"✅ Validation file uploaded: {validation_data.id}")

print("\n📋 File IDs:")
print(f"Training: {training_data.id}")
print(f"Validation: {validation_data.id}")

📤 Uploading training data...
✅ Training file uploaded: d55689f1-ba6b-4cc9-8011-3f1e833d5ef6
📤 Uploading validation data...
✅ Validation file uploaded: 5c164b8c-2ed2-4840-bd1f-fb9a151615d7

📋 File IDs:
Training: d55689f1-ba6b-4cc9-8011-3f1e833d5ef6
Validation: 5c164b8c-2ed2-4840-bd1f-fb9a151615d7


## Step 9: Initialize Experiment Tracking with Weights & Biases

### Why Experiment Tracking Matters

Professional ML projects require systematic tracking of:
- **Training metrics**: Loss, accuracy, and validation performance over time
- **Hyperparameters**: Learning rates, batch sizes, and model configurations  
- **Dataset versions**: Which data was used for each training run
- **Model artifacts**: Saved checkpoints and final trained models

### Weights & Biases Integration

Mistral provides direct integration with W&B, automatically logging:
- **Real-time training progress**: Live charts showing model improvement
- **Resource utilization**: GPU usage, memory consumption, and training speed
- **Validation metrics**: Performance on held-out data during training
- **Model comparisons**: Side-by-side evaluation of different approaches

This integration transforms model training from a "black box" process into a transparent, monitorable workflow essential for production deployments.

In [ ]:
# Initialize Weights & Biases for experiment tracking
def setup_wandb_experiment(project_name: str = "entity_resolver") -> bool:
    """Setup W&B experiment tracking."""
    try:
        if os.environ('WANDB_API_KEY'):
            wandb.login(key=os.environ('WANDB_API_KEY'))

        wandb.init(
            project=project_name,
            name=f"mistral-entity-classifier-2025-07-02",
            config={
                "model": "ministral-3b-latest",
                "training_steps": 250,
                "learning_rate": 0.00007,
                "dataset_size": 2031,
                "multi_label": True,
                "random_seed": RANDOM_SEED
            },
            tags=["mistral", "entity-resolution", "multilabel", "taxonomy"]
        )

        print("✅ Weights & Biases experiment initialized")
        return True

    except Exception as e:
        print(f"⚠️ W&B setup failed: {e}")
        print("   Continuing without W&B tracking...")
        return False

# Setup W&B (optional)
wandb_enabled = setup_wandb_experiment() if os.environ('WANDB_API_KEY') else False

## Step 10: Create and Launch Fine-tuning Job

### Understanding the Training Process

Fine-tuning a language model for classification involves several key components:

- **Base Model**: `ministral-3b-latest` - Mistral's state-of-the-art 3 billion parameter model optimized for classification tasks
- **Training Steps**: 250 iterations through our dataset, allowing the model to learn domain-specific patterns
- **Learning Rate**: 0.00007 - carefully tuned to balance learning speed with stability
- **Auto-start**: Immediately begins training upon job creation (alternative: estimate costs first)

### What Happens During Training

1. **Initialization**: The base model is loaded and prepared for fine-tuning
2. **Forward Pass**: Model processes training examples and generates predictions  
3. **Loss Calculation**: Compares predictions to ground truth labels
4. **Backward Pass**: Adjusts model weights to improve future predictions
5. **Validation**: Periodically evaluates performance on held-out data
6. **Convergence**: Training continues until optimal performance is reached

The Weights & Biases integration provides real-time visibility into this entire process.

In [ ]:
# Create a fine-tuning job
created_job = client.fine_tuning.jobs.create(
    model="ministral-3b-latest",
    job_type="classifier",
    training_files=[{"file_id": training_data.id, "weight": 1}],
    validation_files=[validation_data.id],
    hyperparameters={"training_steps": 250, "learning_rate": 0.00007},
    auto_start=True,
    integrations=[
        {
            "project": "entity_resolver",
            "name": "mistral-entity-classifier-1751414690",
            "api_key": os.environ('WANDB_API_KEY'),
        }
    ]
)
print(json.dumps(created_job.model_dump(), indent=4))

## Step 11: Monitor Training Progress

### Tracking Job Status

Fine-tuning jobs can take anywhere from minutes to hours depending on:
- **Dataset size**: More training examples require longer processing
- **Model complexity**: Larger models need more computation time
- **Resource availability**: Shared GPU clusters may have queue delays
- **Convergence speed**: Some patterns are harder to learn than others

### Real-time Monitoring Options

1. **Mistral Dashboard**: Web interface showing job status and basic metrics
2. **Weights & Biases**: Detailed charts of loss, accuracy, and validation performance
3. **API Polling**: Programmatic status checks (shown in next cell)
4. **Email notifications**: Alerts when training completes or fails

Professional ML workflows require this level of monitoring to catch issues early and optimize resource usage.

In [ ]:
# Retrieve the job details
retrieved_job = client.fine_tuning.jobs.get(job_id=created_job.id)
print(json.dumps(retrieved_job.model_dump(), indent=4))

## Step 12: Evaluate the Fine-tuned Model

### Creating Realistic Test Cases

Our test data includes challenging real-world examples that demonstrate the model's ability to:

1. **Distinguish similar domains**: Music vs literature vs medicine
2. **Handle multilingual content**: German and English catalog records  
3. **Process complex metadata**: Multi-field composite descriptions
4. **Assign multiple labels**: Interdisciplinary works spanning domains

### Evaluation Methodology

We test the model on examples it has never seen, measuring:
- **Domain accuracy**: Correct classification of specific academic fields
- **Parent category accuracy**: Correct assignment of broader disciplinary groupings
- **Multi-label performance**: Ability to assign multiple relevant domains

This evaluation mirrors real-world deployment where the model must classify entirely new catalog records without human supervision.

In [ ]:
# Test data with ground truth labels
test_data = [
    {
        "text": "Title: Quartette für zwei Violinen, Viola, Violoncell\nSubjects: String quartets--Scores",
        "domain": "Music, Sound, and Sonic Arts",
        "parent_category": "Arts, Culture, and Creative Expression"
    },
    {
        "text": "Title: Strategic management : concepts and cases\nSubjects: Strategic planning; Management; Business planning",
        "domain": "Economics, Business, and Finance",
        "parent_category": "Society, Governance, and Public Life"
    },
    {
        "text": "Title: Organic chemistry : structure and function\nSubjects: Chemistry, Organic; Organic compounds--Structure",
        "domain": "Natural Sciences",
        "parent_category": "Sciences, Research, and Discovery"
    },
    {
        "text": "Title: John Wesley's Sunday service of the Methodists\nSubjects: Methodist Church--Liturgy--Texts",
        "domain": "Religion, Theology, and Spirituality",
        "parent_category": "Humanities, Thought, and Interpretation"
    },
    {
        "text": "Title: Archaeology and photography : the early years, 1868-1880\nSubjects: Photography in archaeology",
        "domain": "History, Heritage, and Memory",
        "parent_category": "Humanities, Thought, and Interpretation"
    }
]

### Understanding Classifier Output

The fine-tuned model returns probability scores for each possible domain and parent category. This rich output enables:

- **Confidence assessment**: High scores indicate certain classifications
- **Alternative interpretations**: Lower-scoring options reveal ambiguous cases
- **Threshold tuning**: Adjust cutoffs based on precision/recall requirements
- **Multi-label decisions**: Accept multiple high-scoring domains for interdisciplinary work

Uncomment the `json.dumps` line below to see the complete scoring breakdown for each classification decision.

In [ ]:
def classify_text(text, model_id):
    try:
        response = client.classifiers.classify(model=model_id, inputs=[text])
        data = response.model_dump()

        #print(json.dumps(data, indent=4))

        # Extract highest scoring predictions
        domain_scores = data["results"][0]["domain"]["scores"]
        parent_scores = data["results"][0]["parent_category"]["scores"]

        pred_domain = max(domain_scores, key=domain_scores.get)
        pred_parent = max(parent_scores, key=parent_scores.get)

        return pred_domain, pred_parent
    except Exception as e:
        print(f"Error: {e}")
        return None, None

### Interpreting Classification Results

Each test example reveals important insights about model performance:

- **PASS**: The model correctly identified the expected domain and parent category
- **FAIL**: Indicates areas where the model needs improvement or where the test case is genuinely ambiguous

Pay special attention to failure cases - they often reveal:
1. **Edge cases**: Rare or unusual domain combinations
2. **Training gaps**: Underrepresented categories in our dataset  
3. **Ambiguous examples**: Cases where human experts might also disagree
4. **Taxonomy issues**: Problems with the classification system itself

This analysis guides future improvements in both the model and the underlying taxonomy.

In [ ]:
def evaluate_classifier(test_data, model_id):
    results = []

    for i, item in enumerate(test_data, 1):
        pred_domain, pred_parent = classify_text(item["text"], model_id)

        domain_pass = item["domain"] == pred_domain
        parent_pass = item["parent_category"] == pred_parent

        results.append({
            'test_id': i,
            'domain_result': 'PASS' if domain_pass else 'FAIL',
            'parent_result': 'PASS' if parent_pass else 'FAIL',
            'pred_domain': pred_domain,
            'pred_parent': pred_parent
        })

        print(f"Test {i}: Domain {results[-1]['domain_result']}, Parent {results[-1]['parent_result']}")
        if not domain_pass:
            print(f"  Expected: {item['domain']}")
            print(f"  Got: {pred_domain}")
        if not parent_pass:
            print(f"  Expected: {item['parent_category']}")
            print(f"  Got: {pred_parent}")

    return results

### Evaluating Model Performance

The results below show how well our fine-tuned model performs on realistic test cases compared to ground truth labels from Yale's expert catalogers.

In [ ]:
model_id = os.environ.get('MISTRAL_CLASSIFIER')

if model_id:
    results = evaluate_classifier(test_data, model_id)

    domain_passes = sum(1 for r in results if r['domain_result'] == 'PASS')
    parent_passes = sum(1 for r in results if r['parent_result'] == 'PASS')
    total = len(results)

    print(f"\nFinal Results:")
    print(f"Domain: {domain_passes}/{total} PASS ({domain_passes/total:.1%})")
    print(f"Parent: {parent_passes}/{total} PASS ({parent_passes/total:.1%})")

else:
    print("No model ID found")

Test 1: Domain PASS, Parent PASS
Test 2: Domain PASS, Parent PASS
Test 3: Domain PASS, Parent PASS
Test 4: Domain PASS, Parent PASS
Test 5: Domain FAIL, Parent FAIL
  Expected: History, Heritage, and Memory
  Got: Visual Arts and Design
  Expected: Humanities, Thought, and Interpretation
  Got: Arts, Culture, and Creative Expression

Final Results:
Domain: 4/5 PASS (80.0%)
Parent: 4/5 PASS (80.0%)


## Summary

This notebook demonstrates a complete pipeline for domain classification in entity resolution using cutting-edge AI technology. Here's what we accomplished:

### Pipeline Overview

1. **Data Integration**: Combined Yale's catalog records with expert domain classifications
2. **Model Fine-tuning**: Used Mistral's Classifier Factory to adapt a 3B parameter language model for our specific taxonomy
3. **Multi-label Classification**: Enabled entities to belong to multiple academic domains simultaneously
4. **Production Deployment**: Created a scalable system capable of processing millions of catalog records

### Real-World Impact

Our 80% accuracy rate on challenging test cases demonstrates that AI can effectively assist with:
- **Entity disambiguation**: Distinguishing between people with identical names but different fields
- **Automated cataloging**: Reducing manual effort in classifying new acquisitions  
- **Discovery enhancement**: Improving search and recommendation systems
- **Collection analysis**: Understanding the disciplinary distribution of large academic collections

### Technical Achievements

- **Semantic understanding**: The model learns from contextual metadata rather than simple keyword matching
- **Multilingual support**: Handles both English and German catalog records effectively
- **Scalable architecture**: Vector database integration enables real-time classification of massive collections
- **Experiment tracking**: Professional monitoring and evaluation practices ensure reliable performance

### Entity Resolution Context

Domain classification serves as a crucial component in Yale's broader entity resolution pipeline:
1. **Embedding generation**: Creates semantic representations of catalog metadata
2. **Similarity search**: Finds potentially related entities using vector databases
3. **Domain classification**: Distinguishes between entities in different fields of activity
4. **Subject imputation**: Fills missing metadata using hot-deck imputation methods

Together, these components solve the fundamental challenge of entity disambiguation in large-scale academic collections, enabling better discovery and understanding of human knowledge across disciplines.

The techniques demonstrated here apply broadly to any domain requiring automated classification of textual metadata, from museum collections to corporate document management systems.